# Teorema de Divergente (Gauss)

o Teorema do Divergente liga integrais de superfície de campos de vetores a triplas integrais através da divergência.

**Teorema:**

Seja E uma região sólida simples e seja S a superfície fronteira de E, orientada positivamente (para fora). Seja $\vec F$ um campo vetorial cujas funções componentes tenham derivadas parciais continuas em uma região aberta que contenha E. Então
$$
\iint_S \vec F \cdot \text{d} \vec S = \iiint_E \text{div} \vec F \; \text{d}V
$$
Portanto, o Teorema do Divergente afirma que, sob as condições dadas, o fluxo de $\vec F$ pela fronteira de E é igual à integral tripla da divergência de $\vec F$ em E.

## Preamble

In [10]:
import sympy as sp
import numpy as np
import sympy.vector as sv
import plotly.graph_objects as go
import pandas as pd
import av_utils as av


sp.init_printing()

R = sv.CoordSys3D('R')

t,s,x,y,z, rho, theta, phi,r = sp.symbols('t s x y z rho theta phi r', real=True)

## Exemplo: 

Determine o fluxo do campo vetorial $\vec F(x,y,z) = z \hat i + y \hat j + x \hat k$ sobre a unidade esferica $x^2 + y^2 + z^2 = 1$

**Solução:**

Primeiro calculamos o divergente do campo vetoril $\vec F$

In [2]:
def field(x,y,z):
    return z*R.i + y*R.j + x*R.k

In [4]:
div = av.divergence(field(x,y,z), (x,y,z));

div

A esfera unitária S é a fronteira da bola unitária B dada por $x^2 + y^2 + z^2 = 1$. Então o teorema de Gauss dá o fluxo como
$$
\iint_S \vec F \cdot \text{d}\vec S = \iiint_B \text{div}\vec F \; \text{d}V = \iiint_B 1 \text{d}V = \frac{4 \pi}{3} (1)^3
$$

A integral acima podia ser resolvida no Sympy na seguinte forma. 

In [7]:
def sphere(theta, phi):
    rx = sp.cos(theta)*sp.sin(phi)*R.i
    ry = sp.sin(theta)*sp.sin(phi)*R.j
    rz = sp.cos(phi)*R.k
    return rx+ry+rz

sphere(theta,phi)

In [14]:
sp.integrate(div*(r**2)*sp.sin(phi), (theta,0,2*sp.pi),(phi,0,sp.pi),(r,0,1))

## Elemento de volume

Elemento de volume em sistema de coordenadas cartesianas,
cilíndricas e esféricas são

- cartesiana: $\text{d}V = \text{d}x\text{d}y\text{d}z$
- cilindrica: $\text{d}V = \rho \text{d}\rho\text{d}\theta\text{d}z$
- esferica: $\text{d}V = r^2 \sin(\phi)\text{d}r \text{d}\theta \text{d}\phi$

## Exemplo:

Utilize o teorema de Gauss para calcular $\iint_S \vec F \cdot \text{d}\vec S$, onde $\vec F(x,y,z) = x y \hat i + (y^2 + \text{e}^{x z^2}) \hat j + \sin(x y) \hat k$ e $S$ é a superficie da região $E$ delimitada pelo cilindro parabolico $z=1-x^2$ e os planos $z=0$, $y=0$ e $y+z=2$.

**Solução:**



In [18]:
def cilinder(x,y):
    return x*R.i + y*R.j + (1-x**2)*R.k

cilinder(x,y)

In [19]:
def plane(x,y):
    return x*R.i+y*R.j+(2-y)*R.k

plane(x,y)

In [27]:
pl = av.plot3d_parametric_surface(plane(x,y),(x,-2,3), (y,-2,3))
av.plot3d_parametric_surface(cilinder(x,y), (x,-1,2),(y,-1,2), fig = pl)

In [15]:
def F(x,y,z):
    return x*y*R.i + (y**2 + sp.exp(x*(z**2)))*R.j + sp.sin(x*y)*R.k

F(x,y,z)

In [16]:
div = av.divergence(F(x,y,z),(x,y,z))

div

In [37]:
def plot3d_parametric_surface(func, inter1 = None, inter2 = None, fig = False, xtitle = 'X', 
                                ytitle= 'Y', ztitle = "Z", title='3D Surface Plot', 
                                points = 50, scene_aspectmode = 'data', surfacecolor=None,  showscale=True):

    '''
    `func`: must be a either a tuple with three components or a parametric equation in the class sympy.vector
    `inter1`: (parameter, start, end)
    `inter2`: (parameter, start, end)
    `scene_aspectmode`: string. 'data' or 'cube'
    `surfacecolor`: function. a sympy function to determine the color of the surface. 
    '''
    
    if inter1 is None:
        print("Please input the interval for the first parameter in the format (parameter, begin, end)")
    if inter2 is None:
        print("Please input the interval for the second parameter in the format (parameter, begin, end)")
    
    if isinstance(func, sp.Expr):
        if func.is_Vector:
            R = list(func.separate().keys())[0]
            func_x = func & R.i
            func_y = func & R.j
            func_z = func & R.k
            func = tuple(func.components.values())
    elif isinstance(func, tuple) or isinstance(fiefuncld, list):
        assert len(func)==3, "the field must have three elements"
        func_x = func[0]
        func_y = func[1]
        func_z = func[2]
    else:
        print("the field isn't recognized. it must be a tuple of three sympy functions or a vector field of class sympy.vector")
    #check if the parametric equation has three components.        
    #assert len(func) ==3, 'The parametric equation of a 3D surface must has 3 components.'

    #check if the parameters of the equation are the same as parameters declared in the intervals.
    params = [func[i].free_symbols for i in range(len(func))]
    params_unique = set([item for sublist in params for item in sublist])
    assert params_unique == set([inter1[0],inter2[0]]), "The parameters of the function aren't the same as the ones declared in the intervals"
    
    
    xx_np = sp.lambdify([inter1[0],inter2[0]], func_x)
    yy_np = sp.lambdify([inter1[0],inter2[0]], func_y)
    zz_np = sp.lambdify([inter1[0],inter2[0]], func_z)
    
    var1,var2 = np.linspace(inter1[1],inter1[2],points), np.linspace(inter2[1],inter2[2],points)
    uGrid, vGrid = np.meshgrid(var1, var2)
    xx, yy, zz = xx_np(uGrid,vGrid), yy_np(uGrid,vGrid), zz_np(uGrid,vGrid)
    
    # if one of the coordinate be constant. 
    if isinstance(xx,int):
        xx = xx*np.ones((points,points))
    if isinstance(yy,int):
        yy = yy*np.ones((points,points))
    if isinstance(zz,int):
        zz = zz*np.ones((points,points))
    
    if surfacecolor:
        x_col = np.linspace(xx.min(), xx.max(), points)
        y_col = np.linspace(yy.min(), yy.max(), points)
        z_col = np.linspace(zz.min(), zz.max(), points)
        
        xx_col, yy_col, zz_col = np.meshgrid(x_col,y_col, z_col)
        
        vars = list(surfacecolor.free_symbols)
        color_np = sp.lambdify(vars, surfacecolor) 
        for ind, var in enumerate(vars):
            if var.name == 'x':
                vars[ind] = xx
            elif var.name == 'y':
                vars[ind] = yy
            elif var.name == 'z':
                vars[ind] = zz
        surfacecolor = color_np(*vars)

        
    if fig is False:
        fig = go.Figure()
        fig.add_surface(x = xx , y = yy, z = zz, showlegend=False, surfacecolor = surfacecolor, showscale=showscale)
        fig.update_layout(title=title, xaxis_title=xtitle,
                          yaxis_title= ytitle, scene_aspectmode=scene_aspectmode)
    
    else:
        fig.add_surface(x = xx , y = yy, z = zz, showlegend=False, surfacecolor = surfacecolor, showscale=showscale)
        fig.update_layout(title=title, xaxis_title=xtitle,
                          yaxis_title= ytitle, scene_aspectmode=scene_aspectmode)
    return fig


In [38]:
plot3d_parametric_surface(x*R.i+y*R.j+0*R.k,(x,-2,2), (y,-2,2))